In [10]:
import sys
sys.path.insert(1, '/home/ubuntu/gechen/software/burstfit/examples/')
import import_ipynb
import BurstFit_paper_template as paper_fit

from burstfit.utils.functions import pulse_fn, sgram_fn_vec, sgram_fn, gauss, gauss_norm, model_free_4, model_free_normalized_4, power_law # pulse_fn_vec  
from burstfit.utils.plotter import plot_fit_results
import numpy as np


def real_time_burstfit(candidate, datestring, beam, corr, snr_heimdall, dm_heimdall, width_heimdall,
                       voltage = False, dedisperse = True, mask_chans=[], nfreq = 4, save_plot = True, 
                       plot = False
                      ):
    
    fil_file_dedispersed = '/home/ubuntu/gechen/software/burstfit/fil_files_dedispersed/%s_%d_%s.txt'%(candidate, nfreq, str(voltage))
    
    bd_heimdall, fil_file_heimdall  = paper_fit.prepare_bd(candidate, dm_heimdall, 
                                                           width_heimdall, snr_heimdall, 
                                                           datestring=datestring, beam=beam, 
                                                           corr=corr, voltage=False)


    data_burst, i_low, i_high = paper_fit.prepare_burst_data_heimdall_only(fil_file_heimdall, fil_file_dedispersed, 
                                                        candidate, bd_heimdall, 
                                                       t_chop_width = 5, dedisperse=dedisperse,
                                                                          plot = plot)
    
    profile_bounds=[(0, 0.5 / bd_heimdall.tsamp - i_low - bd_heimdall.width, 0, 0),(np.inf, 0.5 / bd_heimdall.tsamp - i_low + bd_heimdall.width, 5 * bd_heimdall.width, np.inf)]

    bf_S1T2_c1 = paper_fit.fit_paper_curvefit(bd_heimdall, data_burst, ['S', 'mu_t', 'sigma_t', 'tau'], pulse_fn, 
              profile_bounds, 
              ['c0', 'c1', 'c2'], model_free_normalized_4, fix_ncomp = True, 
                                              ncomp = 1, plot = plot)
    
    dm_fit, dm_fit_err = bf_S1T2_c1.sgram_params['all'][1]['popt'][-1], bf_S1T2_c1.sgram_params['all'][1]['perr'][-1]

    width_samp, width_samp_fine_pow2 = paper_fit.bin_best(bf_S1T2_c1, bd_heimdall)
    bin_num = paper_fit.bin_number(bf_S1T2_c1, bd_heimdall, i_low, width_samp_fine_pow2)
    print("width_samp, width_samp_fine_pow2, bin_num = ", width_samp, width_samp_fine_pow2, bin_num)

    data_new_dm, dm_fit = paper_fit.Dedisperse_data_profile(bf_S1T2_c1, fil_file_heimdall, width_samp)

    best_bin_snr = paper_fit.Compare_bins(data_new_dm, bd_heimdall, width_samp_fine_pow2, plot = plot)

    dict_burstfit = {
        "bf1_width_bins": width_samp_fine_pow2, # already converted to voltage resolution 
        "bf1_start_bins": bin_num, # already converted to voltage resolution
        "bf1_dm":dm_fit, 
        "bf1_dm_stddev": dm_fit_err,
        "bf1_reduced_chisq": bf_S1T2_c1.reduced_chi_sq,
        "bf1_pvalue": bf_S1T2_c1.p_value
    }
    
    if save_plot:
        plot_fit_results(bf_S1T2_c1.sgram, bf_S1T2_c1.sgram_model.evaluate, bf_S1T2_c1.sgram_params['all'][1]['popt'], 
                          bf_S1T2_c1.tsamp, bf_S1T2_c1.fch1, bf_S1T2_c1.foff, show=True, save=True, 
                         outname = 'bf1_' + candidate, 
                         outdir='/dataz/dsa110/operations/candidates/' + candidate + '/other')        
    
    return dict_burstfit


    




In [12]:
candidate = '220330aaan'
datestring = '2022_3_29_23_4_54'
beam = 164
corr = 'corr09'
snr_heimdall = 12.9
dm_heimdall = 467.8
width_heimdall = 32 

# '220330aaan', '2022_3_29_23_4_54', 164, 'corr09', 12.9, 467.8, 32  

# candidate = sys.argv[0]
# datestring = sys.argv[1]
# beam = sys.argv[2]
# corr = sys.argv[3]
# snr_heimdall = sys.argv[4]
# dm_heimdall = sys.argv[5]
# width_heimdall = sys.argv[6]

real_time_burstfit(candidate, datestring, beam, corr, snr_heimdall, dm_heimdall, width_heimdall, save_plot=False)

2022-06-29 20:44:23,187 - prepare_data -burstfit.data - INFO - Preparing data for burst fitting.
2022-06-29 20:44:23,395 - crop_dedispersed_data -burstfit.data - INFO - Cropping data with time_window: 0.2s.
2022-06-29 20:44:23,398 - normalise_data -burstfit.data - INFO - Normalising data using off pulse mean and std.
2022-06-29 20:44:23,460 - normalise_data -burstfit.data - INFO - Off pulse mean and std are: (36.063605401529635, 21.164850863543236)


using filterbank  /data/dsa110/T1/corr09/2022_3_29_23_4_54/fil_220330aaan/220330aaan_164.fil
Only using Heimdall filterbank.
0.5 0.000262144 5 32 1747 2067
1747 2067 320 (4, 320)
self.function_input_names:  ['S', 'mu', 'sigma', 'tau']
self.function_input_names:  ['c0', 'c1', 'c2']


2022-06-29 20:44:24,409 - run_tests -burstfit.fit - INFO - Running statistical tests on the residual.
2022-06-29 20:44:24,410 - run_tests -burstfit.fit - INFO - Running off pulse - off pulse test
2022-06-29 20:44:24,414 - tests -root - INFO - P values: T-test (0.94271), Kruskal (0.90986), KS (0.77281), F-test (0.09014)
2022-06-29 20:44:24,415 - run_tests -burstfit.fit - INFO - Running on pulse - off pulse (L) test
2022-06-29 20:44:24,419 - tests -root - INFO - P values: T-test (0.00000), Kruskal (0.00000), KS (0.00000), F-test (0.00000)
2022-06-29 20:44:24,420 - run_tests -burstfit.fit - INFO - Running on pulse - off pulse (R) test
2022-06-29 20:44:24,424 - tests -root - INFO - P values: T-test (0.00000), Kruskal (0.00000), KS (0.00000), F-test (0.00003)
2022-06-29 20:44:24,426 - fitall -burstfit.fit - WARNING - Input profile bounds detected. Using them for component 1
2022-06-29 20:44:24,428 - fitcycle -burstfit.fit - INFO - Fitting component 1.
2022-06-29 20:44:24,429 - initial_profi

<function model_free_normalized_4 at 0x7fe3f689b488>
[0.39878002 0.23669735 0.47927672 0.16927181]


2022-06-29 20:44:24,544 - initial_spectrafit -burstfit.fit - INFO - Converged parameters (spectra fit) are:
2022-06-29 20:44:24,546 - initial_spectrafit -burstfit.fit - INFO - c0: 0.3987800180912018 +- 0.0
2022-06-29 20:44:24,547 - initial_spectrafit -burstfit.fit - INFO - c1: 0.23669734597206116 +- 0.0
2022-06-29 20:44:24,549 - initial_spectrafit -burstfit.fit - INFO - c2: 0.47927671670913696 +- 0.0
2022-06-29 20:44:24,551 - sgram_fit -burstfit.fit - INFO - Running sgram profile fit for component: 1
2022-06-29 20:44:24,552 - sgram_fit -burstfit.fit - INFO - initial estimate for parameters: [0.39878002, 0.23669735, 0.47927672, 190.51959045069415, 161.3835343097292, 12.843949515958855, 1.0, 467.8]
2022-06-29 20:44:24,598 - run_fit -burstfit.curvefit - WARNING - Fit errors are not finite. Retrying with p0+-(0.2*p0) bounds
2022-06-29 20:44:25,231 - sgram_fit -burstfit.fit - INFO - Converged parameters are:
2022-06-29 20:44:25,235 - sgram_fit -burstfit.fit - INFO - c0: 0.3819557881701074 +

{bf}.calc_redchisq()= (1.2300534166419113, 2.9571803894512527e-08)
width_samp, width_samp_fine_pow2, bin_num =  31 128 119
64
128
256
512
1024
fine resolution bin size (at 32.768e-6s) that gives the max snr: 256


{'bf1_width_bins': 128,
 'bf1_start_bins': 119,
 'bf1_dm': 474.52522988553045,
 'bf1_dm_stddev': 1.6737556598976506,
 'bf1_reduced_chisq': 1.2300534166419113,
 'bf1_pvalue': 2.9571803894512527e-08}